In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode,hash,array

spark=SparkSession.builder.appName("negotiated_tare").config("spark.driver.memory","4g").getOrCreate()


25/04/28 14:11:47 WARN Utils: Your hostname, aayushgyawali resolves to a loopback address: 127.0.1.1; using 10.10.42.111 instead (on interface enp2s0)
25/04/28 14:11:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/28 14:11:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:

# df = spark.read.option("multiline",True).json("innetwork.json")
# _schema=billing_code string

In [3]:
df = spark.read.option("multiline",True).json("innetwork.json")
df.printSchema()

root
 |-- billing_code: string (nullable = true)
 |-- billing_code_type: string (nullable = true)
 |-- billing_code_type_version: string (nullable = true)
 |-- description: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negotiated_rates: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- negotiated_prices: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- additional_information: string (nullable = true)
 |    |    |    |    |-- billing_class: string (nullable = true)
 |    |    |    |    |-- billing_code_modifier: array (nullable = true)
 |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |-- expiration_date: string (nullable = true)
 |    |    |    |    |-- negotiated_rate: double (nullable = true)
 |    |    |    |    |-- negotiated_type: string (nullable = true)
 |    |    |    |    |-- service_code: array (nullable = true)
 |    |    |

In [4]:
negotiated_rates_df =df.withColumn("row1",explode("negotiated_rates"))
negotiated_prices_df = negotiated_rates_df.withColumn("row2",explode("row1.negotiated_prices"))

rate_df = negotiated_prices_df.select("billing_code","billing_code_type","negotiation_arrangement",
                                      col("row2.billing_code_modifier").alias("billing_code_modifier"),
                                      col("row2.billing_class").alias("billing_class"),
                                      col("row2.negotiated_rate").alias("negotiated_rate"),
                                      col("row2.service_code").alias("service_code"),
                                      col("row1.provider_references").alias("provided_group_id"),
                                      col("row2.negotiated_type").alias("negotiated_type"))
rate_df.show()

25/04/28 14:12:01 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
                                                                                

+------------+-----------------+-----------------------+---------------------+-------------+---------------+------------+--------------------+---------------+
|billing_code|billing_code_type|negotiation_arrangement|billing_code_modifier|billing_class|negotiated_rate|service_code|   provided_group_id|negotiated_type|
+------------+-----------------+-----------------------+---------------------+-------------+---------------+------------+--------------------+---------------+
|       PEMG1|              CPT|                    ffs|                 NULL| professional|           70.0|        [11]|[10001001, 100020...|     negotiated|
|       PEMG1|              CPT|                    ffs|                 NULL| professional|           77.0|        [11]|          [10048001]|     negotiated|
|       PEMG1|              CPT|                    ffs|                 NULL| professional|         119.97|        [11]|          [10077001]|     negotiated|
|       PEMG1|              CPT|              

## Hashing service_code

In [5]:
hashing_df=rate_df.withColumn("service_code",hash("service_code"))
hashing_df.show()
hashing_df.printSchema()

[Stage 2:>                                                          (0 + 1) / 1]

+------------+-----------------+-----------------------+---------------------+-------------+---------------+------------+--------------------+---------------+
|billing_code|billing_code_type|negotiation_arrangement|billing_code_modifier|billing_class|negotiated_rate|service_code|   provided_group_id|negotiated_type|
+------------+-----------------+-----------------------+---------------------+-------------+---------------+------------+--------------------+---------------+
|       PEMG1|              CPT|                    ffs|                 NULL| professional|           70.0| -1726485189|[10001001, 100020...|     negotiated|
|       PEMG1|              CPT|                    ffs|                 NULL| professional|           77.0| -1726485189|          [10048001]|     negotiated|
|       PEMG1|              CPT|                    ffs|                 NULL| professional|         119.97| -1726485189|          [10077001]|     negotiated|
|       PEMG1|              CPT|              

## Rename

[Stage 3:>                                                          (0 + 1) / 1]

+-----+---+----+----+------------+------+-----------+--------------------+----------+
|   bC|bCT|negA| mdH|        bCls|  negR|       poSH|   provided_group_id|      negT|
+-----+---+----+----+------------+------+-----------+--------------------+----------+
|PEMG1|CPT| ffs|NULL|professional|  70.0|-1726485189|[10001001, 100020...|negotiated|
|PEMG1|CPT| ffs|NULL|professional|  77.0|-1726485189|          [10048001]|negotiated|
|PEMG1|CPT| ffs|NULL|professional|119.97|-1726485189|          [10077001]|negotiated|
|PEMG1|CPT| ffs|NULL|professional|  94.5|-1726485189|          [10087001]|negotiated|
|PEMG1|CPT| ffs|NULL|professional|141.51|-1726485189|[10152001, 10153001]|negotiated|
|PEMG1|CPT| ffs|NULL|professional|  84.0|-1726485189|[10187001, 10395001]|negotiated|
|PEMG1|CPT| ffs|NULL|professional| 106.4|-1726485189|          [10259001]|negotiated|
|PEMG1|CPT| ffs|NULL|professional| 108.5|-1726485189|          [10286001]|negotiated|
|PEMG1|CPT| ffs|NULL|professional| 105.0|-1726485189|[

In [7]:
renamecol_df.printSchema()


root
 |-- bC: string (nullable = true)
 |-- bCT: string (nullable = true)
 |-- negA: string (nullable = true)
 |-- mdH: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- bCls: string (nullable = true)
 |-- negR: double (nullable = true)
 |-- poSH: integer (nullable = false)
 |-- provided_group_id: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- negT: string (nullable = true)



In [8]:
rate_df.filter(rate_df.billing_code_modifier.isNotNull()).count()

97508

In [9]:
df_cast = renamecol_df.withColumn("poSH",array(col("poSH")))
df_cast.printSchema()

root
 |-- bC: string (nullable = true)
 |-- bCT: string (nullable = true)
 |-- negA: string (nullable = true)
 |-- mdH: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- bCls: string (nullable = true)
 |-- negR: double (nullable = true)
 |-- poSH: array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- provided_group_id: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- negT: string (nullable = true)



In [10]:
df_cast.toPandas().to_json('output2.json', orient='records')


In [ ]:
bcls